# Unidad 2.4 - Operaciones Morfológicas

## Objetivos
- Aplicar binarización (threshold simple y Otsu)
- Implementar erosión y dilatación
- Utilizar operaciones compuestas (apertura y cierre)
- Limpiar ruido en imágenes binarias
- Entender el efecto del tamaño del kernel

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Binarización de imágenes

La morfología trabaja principalmente con imágenes binarias (blanco y negro).

In [ ]:
# Cargar imagen en escala de grises
img = cv2.imread("imagenes/DPP0357.TIF", cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Error: No se pudo cargar la imagen")
else:
    print(f"Imagen cargada: {img.shape}")
    
    plt.figure(figsize=(8,6))
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original en Escala de Grises')
    plt.colorbar(label='Intensidad')
    plt.axis('off')
    plt.show()

### 1.1 Threshold Simple (Manual)

In [ ]:
# Threshold simple con diferentes valores
umbrales = [100, 127, 150, 180]

plt.figure(figsize=(15, 4))
for i, umbral in enumerate(umbrales):
    _, binary = cv2.threshold(img, umbral, 255, cv2.THRESH_BINARY)
    
    plt.subplot(1, 4, i+1)
    plt.imshow(binary, cmap='gray')
    plt.title(f'Threshold = {umbral}')
    plt.axis('off')

plt.suptitle('Binarización con Threshold Manual', fontsize=14)
plt.tight_layout()
plt.show()

### 1.2 Threshold de Otsu (Automático)

El método de Otsu calcula automáticamente el umbral óptimo.

In [ ]:
# Threshold de Otsu
umbral_otsu, binary_otsu = cv2.threshold(img, 0, 255, 
                                          cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"Umbral calculado por Otsu: {umbral_otsu:.2f}")

# Comparación
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(binary_otsu, cmap='gray')
plt.title(f'Otsu (umbral = {umbral_otsu:.0f})')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.hist(img.ravel(), bins=256, color='gray', alpha=0.7)
plt.axvline(x=umbral_otsu, color='red', linestyle='--', linewidth=2, label='Umbral Otsu')
plt.title('Histograma')
plt.xlabel('Intensidad')
plt.ylabel('Frecuencia')
plt.legend()

plt.tight_layout()
plt.show()

## 2. Elementos Estructurantes (Kernels)

El kernel define la forma y tamaño del vecindario para las operaciones morfológicas.

In [ ]:
# Crear diferentes tipos de kernels
kernel_rect_3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
kernel_rect_5 = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
kernel_rect_7 = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))

kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))

# Visualizar kernels
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()

kernels = [
    (kernel_rect_3, 'Rectangular 3×3'),
    (kernel_rect_5, 'Rectangular 5×5'),
    (kernel_rect_7, 'Rectangular 7×7'),
    (kernel_ellipse, 'Elíptico 5×5'),
    (kernel_cross, 'Cruz 5×5'),
]

for i, (kernel, titulo) in enumerate(kernels):
    axes[i].imshow(kernel, cmap='gray', interpolation='nearest')
    axes[i].set_title(titulo)
    axes[i].axis('off')
    
    # Mostrar valores del kernel
    for row in range(kernel.shape[0]):
        for col in range(kernel.shape[1]):
            axes[i].text(col, row, str(kernel[row, col]), 
                        ha='center', va='center', color='red', fontsize=9)

# Ocultar el último subplot
axes[5].axis('off')

plt.suptitle('Tipos de Elementos Estructurantes', fontsize=14)
plt.tight_layout()
plt.show()

print("\nKernel rectangular 5×5:")
print(kernel_rect_5)

## 3. Erosión

La erosión reduce el tamaño de los objetos blancos y elimina ruido pequeño.

In [ ]:
# Usar la imagen binaria de Otsu
binary = binary_otsu.copy()

# Crear kernel para morfología
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

# Aplicar erosión con diferentes iteraciones
erosion_1 = cv2.erode(binary, kernel, iterations=1)
erosion_2 = cv2.erode(binary, kernel, iterations=2)
erosion_3 = cv2.erode(binary, kernel, iterations=3)
erosion_5 = cv2.erode(binary, kernel, iterations=5)

# Visualización
plt.figure(figsize=(15, 8))

imagenes_erosion = [
    (binary, 'Original'),
    (erosion_1, 'Erosión iter=1'),
    (erosion_2, 'Erosión iter=2'),
    (erosion_3, 'Erosión iter=3'),
    (erosion_5, 'Erosión iter=5')
]

for i, (imagen, titulo) in enumerate(imagenes_erosion):
    plt.subplot(2, 3, i+1)
    plt.imshow(imagen, cmap='gray')
    plt.title(titulo)
    plt.axis('off')

plt.suptitle('Efecto de la Erosión (kernel 5×5)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Dilatación

La dilatación expande el tamaño de los objetos blancos y conecta regiones cercanas.

In [ ]:
# Aplicar dilatación con diferentes iteraciones
dilatacion_1 = cv2.dilate(binary, kernel, iterations=1)
dilatacion_2 = cv2.dilate(binary, kernel, iterations=2)
dilatacion_3 = cv2.dilate(binary, kernel, iterations=3)
dilatacion_5 = cv2.dilate(binary, kernel, iterations=5)

# Visualización
plt.figure(figsize=(15, 8))

imagenes_dilatacion = [
    (binary, 'Original'),
    (dilatacion_1, 'Dilatación iter=1'),
    (dilatacion_2, 'Dilatación iter=2'),
    (dilatacion_3, 'Dilatación iter=3'),
    (dilatacion_5, 'Dilatación iter=5')
]

for i, (imagen, titulo) in enumerate(imagenes_dilatacion):
    plt.subplot(2, 3, i+1)
    plt.imshow(imagen, cmap='gray')
    plt.title(titulo)
    plt.axis('off')

plt.suptitle('Efecto de la Dilatación (kernel 5×5)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Comparación Directa: Erosión vs Dilatación

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(erosion_1, cmap='gray')
plt.title('Erosión\n(reduce objetos)')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(binary, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(dilatacion_1, cmap='gray')
plt.title('Dilatación\n(expande objetos)')
plt.axis('off')

plt.tight_layout()
plt.show()

## 6. Apertura (Opening)

Apertura = Erosión + Dilatación

Elimina ruido externo pequeño (puntos blancos aislados).

In [ ]:
# Apertura con cv2.morphologyEx
apertura = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# Hacer apertura manualmente para visualizar los pasos
erosion_temp = cv2.erode(binary, kernel, iterations=1)
apertura_manual = cv2.dilate(erosion_temp, kernel, iterations=1)

# Visualización paso a paso
plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(binary, cmap='gray')
plt.title('1. Original')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(erosion_temp, cmap='gray')
plt.title('2. Erosión')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(apertura_manual, cmap='gray')
plt.title('3. Dilatación\n(Apertura manual)')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(apertura, cmap='gray')
plt.title('4. cv2.MORPH_OPEN\n(equivalente)')
plt.axis('off')

plt.suptitle('Apertura: Erosión + Dilatación', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Cierre (Closing)

Cierre = Dilatación + Erosión

Rellena huecos internos pequeños (puntos negros dentro de objetos).

In [ ]:
# Cierre con cv2.morphologyEx
cierre = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# Hacer cierre manualmente para visualizar los pasos
dilatacion_temp = cv2.dilate(binary, kernel, iterations=1)
cierre_manual = cv2.erode(dilatacion_temp, kernel, iterations=1)

# Visualización paso a paso
plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(binary, cmap='gray')
plt.title('1. Original')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(dilatacion_temp, cmap='gray')
plt.title('2. Dilatación')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(cierre_manual, cmap='gray')
plt.title('3. Erosión\n(Cierre manual)')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(cierre, cmap='gray')
plt.title('4. cv2.MORPH_CLOSE\n(equivalente)')
plt.axis('off')

plt.suptitle('Cierre: Dilatación + Erosión', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Comparación: Apertura vs Cierre

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(binary, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(apertura, cmap='gray')
plt.title('Apertura\n(elimina ruido externo)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cierre, cmap='gray')
plt.title('Cierre\n(rellena huecos internos)')
plt.axis('off')

plt.tight_layout()
plt.show()

## 9. Ejemplo Práctico: Limpieza de Ruido Sal y Pimienta

In [ ]:
def agregar_ruido_sp(imagen, prob_sal=0.01, prob_pimienta=0.01):
    """
    Agrega ruido sal y pimienta a una imagen binaria.
    
    Parámetros:
        imagen: imagen binaria de entrada
        prob_sal: probabilidad de píxeles blancos (sal)
        prob_pimienta: probabilidad de píxeles negros (pimienta)
    
    Retorna:
        imagen con ruido
    """
    ruidosa = imagen.copy()
    total_pixels = imagen.size
    
    # Sal (píxeles blancos)
    num_sal = int(prob_sal * total_pixels)
    coords_sal = [np.random.randint(0, i, num_sal) for i in imagen.shape]
    ruidosa[coords_sal[0], coords_sal[1]] = 255
    
    # Pimienta (píxeles negros)
    num_pimienta = int(prob_pimienta * total_pixels)
    coords_pimienta = [np.random.randint(0, i, num_pimienta) for i in imagen.shape]
    ruidosa[coords_pimienta[0], coords_pimienta[1]] = 0
    
    return ruidosa

# Crear imagen con ruido
np.random.seed(42)  # Para reproducibilidad
binary_ruidosa = agregar_ruido_sp(binary, prob_sal=0.02, prob_pimienta=0.02)

print("Ruido agregado exitosamente")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(binary, cmap='gray')
plt.title('Original Limpia')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(binary_ruidosa, cmap='gray')
plt.title('Con Ruido Sal y Pimienta (2% cada uno)')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Limpiar el ruido usando morfología
# Paso 1: Apertura para eliminar sal (puntos blancos aislados)
limpia_apertura = cv2.morphologyEx(binary_ruidosa, cv2.MORPH_OPEN, kernel)

# Paso 2: Cierre para eliminar pimienta (puntos negros en objetos)
limpia_completa = cv2.morphologyEx(limpia_apertura, cv2.MORPH_CLOSE, kernel)

# Visualización del proceso de limpieza
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(binary, cmap='gray')
plt.title('1. Original Limpia')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(binary_ruidosa, cmap='gray')
plt.title('2. Con Ruido Sal y Pimienta')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(limpia_apertura, cmap='gray')
plt.title('3. Después de Apertura\n(elimina sal)')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.imshow(limpia_completa, cmap='gray')
plt.title('4. Después de Cierre\n(elimina pimienta)')
plt.axis('off')

# Comparación final
plt.subplot(2, 3, 5)
diferencia = cv2.absdiff(binary, binary_ruidosa)
plt.imshow(diferencia, cmap='hot')
plt.title('Ruido Agregado\n(diferencia)')
plt.axis('off')

plt.subplot(2, 3, 6)
diferencia_final = cv2.absdiff(binary, limpia_completa)
plt.imshow(diferencia_final, cmap='hot')
plt.title('Diferencia Final\n(casi cero)')
plt.axis('off')

plt.suptitle('Limpieza de Ruido con Morfología', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Píxeles diferentes después de limpieza: {np.sum(diferencia_final > 0)}")

## 10. Otras Operaciones Morfológicas

### 10.1 Gradiente Morfológico

Gradiente = Dilatación - Erosión

Resalta los bordes de los objetos.

In [ ]:
# Gradiente morfológico
gradiente = cv2.morphologyEx(binary, cv2.MORPH_GRADIENT, kernel)

# Calcular manualmente para entender
dilatado = cv2.dilate(binary, kernel, iterations=1)
erosionado = cv2.erode(binary, kernel, iterations=1)
gradiente_manual = cv2.subtract(dilatado, erosionado)

plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(binary, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(dilatado, cmap='gray')
plt.title('Dilatación')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(erosionado, cmap='gray')
plt.title('Erosión')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(gradiente, cmap='gray')
plt.title('Gradiente Morfológico\n(Dilatación - Erosión)')
plt.axis('off')

plt.tight_layout()
plt.show()

### 10.2 Top Hat y Black Hat

In [ ]:
# Top Hat: Original - Apertura (objetos pequeños brillantes)
tophat = cv2.morphologyEx(binary, cv2.MORPH_TOPHAT, kernel)

# Black Hat: Cierre - Original (objetos pequeños oscuros)
blackhat = cv2.morphologyEx(binary, cv2.MORPH_BLACKHAT, kernel)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(binary, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(tophat, cmap='gray')
plt.title('Top Hat\n(Original - Apertura)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(blackhat, cmap='gray')
plt.title('Black Hat\n(Cierre - Original)')
plt.axis('off')

plt.tight_layout()
plt.show()

## 11. Comparación de Todas las Operaciones

In [ ]:
# Crear un resumen visual de todas las operaciones
operaciones = {
    'Original': binary,
    'Erosión': cv2.erode(binary, kernel, iterations=1),
    'Dilatación': cv2.dilate(binary, kernel, iterations=1),
    'Apertura': cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel),
    'Cierre': cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel),
    'Gradiente': cv2.morphologyEx(binary, cv2.MORPH_GRADIENT, kernel),
    'Top Hat': cv2.morphologyEx(binary, cv2.MORPH_TOPHAT, kernel),
    'Black Hat': cv2.morphologyEx(binary, cv2.MORPH_BLACKHAT, kernel)
}

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.ravel()

for i, (nombre, imagen) in enumerate(operaciones.items()):
    axes[i].imshow(imagen, cmap='gray')
    axes[i].set_title(nombre, fontsize=12)
    axes[i].axis('off')

# Ocultar el último subplot
axes[8].axis('off')

plt.suptitle('Operaciones Morfológicas (kernel 5×5)', fontsize=14)
plt.tight_layout()
plt.show()

## 12. Efecto del Tamaño del Kernel

In [ ]:
# Probar diferentes tamaños de kernel
sizes = [3, 5, 7, 9, 11]

plt.figure(figsize=(15, 6))

for i, size in enumerate(sizes):
    kernel_temp = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    apertura_temp = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_temp)
    
    plt.subplot(2, 5, i+1)
    plt.imshow(apertura_temp, cmap='gray')
    plt.title(f'Apertura {size}×{size}')
    plt.axis('off')

for i, size in enumerate(sizes):
    kernel_temp = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    cierre_temp = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_temp)
    
    plt.subplot(2, 5, i+6)
    plt.imshow(cierre_temp, cmap='gray')
    plt.title(f'Cierre {size}×{size}')
    plt.axis('off')

plt.suptitle('Efecto del Tamaño del Kernel', fontsize=14)
plt.tight_layout()
plt.show()

## 13. Ejercicio Práctico

Carga tu propia imagen y experimenta con operaciones morfológicas:

In [ ]:
# TODO: Cargar tu imagen
# mi_imagen = cv2.imread("ruta/a/tu/imagen.jpg", cv2.IMREAD_GRAYSCALE)

# TODO: Binarizar
# _, mi_binaria = cv2.threshold(mi_imagen, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# TODO: Aplicar operaciones morfológicas
# 1. Erosión y dilatación
# 2. Apertura y cierre
# 3. Agregar ruido y limpiar
# 4. Probar diferentes tamaños de kernel

print("Completa este ejercicio con tu propia imagen")

## Conclusiones

- **Binarización**: Paso previo esencial (Otsu es automático y robusto)
- **Elemento estructurante**: Define el alcance de las operaciones morfológicas
- **Erosión**: Reduce objetos blancos, elimina ruido pequeño
- **Dilatación**: Expande objetos blancos, conecta regiones
- **Apertura**: Elimina ruido externo (sal)
- **Cierre**: Rellena huecos internos (pimienta)
- **Tamaño del kernel**: Afecta significativamente el resultado
- **Combinación**: Apertura + Cierre es efectiva para limpiar ruido sal y pimienta